In [9]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
from cleantext import clean

In [10]:
arr = os.listdir('.')
print(arr)

['Creation_matrice_Creation.ipynb', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'README.md', 'DataBase_temporary_creation.csv', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Reseau_neurone.ipynb', 'DataElon_clean.csv', 'Vectorisation.ipynb', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [11]:
DataElon = pd.read_csv('DataElon_clean.csv')
DataNONElon = pd.read_csv('DataNONEL_clean.csv')

In [12]:
def cleaner(Database_init):
    Database = Database_init.copy()   
    for i in Database.index:
        Database.loc[i, 'tweet'] = str(Database.loc[i, 'tweet']).lower()
        if '@' in str(Database.loc[i,"tweet"]):
            Tweet = Database.loc[i, 'tweet'].split(' ')
            for index, mot in enumerate(Tweet):
                if '@' in mot:
                    del Tweet[index]
            Database.loc[i, 'tweet'] = ' '.join(Tweet)
        
        Database.loc[i, 'tweet'] = clean(Database.loc[i, 'tweet'], no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
        
        
    return Database

In [13]:
def creation_mot(Liste):
    liste = [' '.join(Liste)]
    Liste_mots_doublon = liste[0].split(' ')
    for index, mot in enumerate(Liste_mots_doublon):
        if '@' in mot:
            Liste_mots_doublon[index] = ''
    Liste_mots = list(set(Liste_mots_doublon))
    return Liste_mots

In [14]:
def creation_matrice(Liste_mots, Liste):
    row =[]
    
    dictionnary = dict()
    for mot in Liste_mots:
        dictionnary[mot] = 0
    
    for tweet in Liste:
        dictionnary2 = dictionnary.copy()
        tweet_actual = tweet.split(" ")
        tweet_actual_clean = []
        for mot_tweet in tweet_actual:
            if mot_tweet != '':
                tweet_actual_clean.append(mot_tweet)
        for index, mot in enumerate(tweet_actual_clean):
            if mot in Liste_mots:
                dictionnary2[mot] = index + 1 
        row.append(dictionnary2)
    return DataFrame(row)

In [15]:
def optimize_matrice(Dataframe, n_min):
    for col in Dataframe.columns:
        if len(Dataframe[Dataframe[col] != 0]) < n_min:
            Dataframe.drop(columns = col, inplace=True)

In [16]:
def normalize(Matrix):
    Matrix_out = Matrix.copy()
    index = Matrix_out.index
    for ind in index:
        Matrix_out.loc[ind] = Matrix_out.loc[ind] / Matrix_out.loc[ind].max()
    return Matrix_out

In [10]:
DataElon = cleaner(DataElon)
Liste_tweets_EM = DataElon["tweet"].values

In [12]:
DataElon = cleaner(DataElon)
Liste_tweets_EM = DataElon["tweet"].values

DataNONElon = cleaner(DataNONElon)
DataNONElon.dropna(inplace=True)
Liste_tweets_NONEM = DataNONElon["tweet"].values

Liste_tweets = np.concatenate((Liste_tweets_EM[8000:], Liste_tweets_NONEM[6001:14001]))
Liste_mots = creation_mot(Liste_tweets)

In [16]:
DataBase_EM = creation_matrice(Liste_mots, Liste_tweets_EM[8000:])
DataBase_EM.drop(columns = '', axis=1, inplace=True)

DataBase_NEM = creation_matrice(Liste_mots, Liste_tweets_NONEM[6001:14001])
DataBase_NEM.drop(columns = '', axis=1, inplace=True)

DataBase = pd.concat([DataBase_EM, DataBase_NEM], ignore_index = True)

In [17]:
DataBase = pd.read_csv("DataBase_temporary_creation.csv")

In [30]:
df = pd.read_csv("DataBase_temporary_creation.csv")
df

,disincent,talktesla,hawking,spain,prolonged,proceeds,skit,pouritup,charter,dmatkins,...,hardware,general,socratic,gourdgeous,mere,resolutions,feathers,gagaahshotel,yard,hotels
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12558,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12559,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
optimize_matrice(DataBase, 18)

In [23]:
DataBase_word = DataBase.copy()

In [24]:
#Rajoute une colonne label : 1 pour un Tweet d'Elon Musk, 0 sinon
DataBase_word['label'] = (DataBase_word.index < 6000).astype(np.int64)

#Supprime les Tweets qui ont des zéros pour toutes les variables
L =[]
for i in DataBase_word.index:
    if DataBase_word.iloc[i, :-1].sum() == 0:
        L.append(i)
DataBase_word.drop(index = L, inplace=True)

In [25]:
DataBase_word.reset_index(drop=True, inplace=True)

In [27]:
Matrice_train = DataBase_word.iloc[:,:-1]
Label_train = DataBase_word.iloc[:,-1:]

In [29]:
Matrice_train

,realize,damn,mins,party,plan,because,announcement,one,sun,lower,...,major,video,ones,meet,they,thinking,outside,company,red,song
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11825,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
DataTrain = normalize(Matrice_train)
DataTrain.to_csv("Data_train_2.csv", index=False)
Label_train.to_csv("Label_train_2.csv", index=False)